# Data Validation


In [2]:
%config IPCompleter.greedy=True

In [7]:
import tensorflow_data_validation as tfdv
import pwd, os


## Generate base directory

In [ ]:
base_dir = pwd.getpwuid(os.getuid()).pw_dir
#base_dir = os.getcwd()
data_dir_str = 'Github/building-machine-learning-pipelines/data'
data_dir = os.path.join(base_dir, data_dir_str)
print(os.path.join(data_dir, 'consumer_complaints.csv'))

###  Generate Statitistics for csv files

In [ ]:

stats = tfdv.generate_statistics_from_csv(data_location=os.path.join(data_dir, 'consumer_complaints_with_narrative.csv'), delimiter=',')

### Generate Statitistics for tensor flow files
This will fail unless you have a tfrecord dataset to include.

In [ ]:
stats = tfdv.generate_statistics_from_tfrecord(data_location=os.path.join(data_dir, 'consumer_complaints_with_narrative.tfrecord'))


## Visualize the statistics

In [15]:
tfdv.visualize_statistics(lhs_statistics=stats)

## Generating a Schema
Generate a schema and display it.

In [16]:
schema = tfdv.infer_schema(stats)
tfdv.display_schema(schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'product',STRING,required,,'product'
'sub_product',STRING,optional,single,'sub_product'
'issue',STRING,required,,'issue'
'sub_issue',STRING,optional,single,'sub_issue'
'consumer_complaint_narrative',BYTES,required,,-
'company',BYTES,required,,-
'state',STRING,optional,single,'state'
'zip_code',BYTES,optional,single,-
'company_response',STRING,required,,'company_response'


/Users/seanwilliams/opt/anaconda3/envs/py38/lib/python3.8/site-packages/tensorflow_data_validation/utils/display_util.py:151: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


,Values
Domain,
'product',"'Bank account or service', 'Consumer Loan', 'Credit card', 'Credit reporting', 'Debt collection', 'Money transfers', 'Mortgage', 'Other financial service', 'Payday loan', 'Prepaid card', 'Student loan'"
'sub_product',"'(CD) Certificate of deposit', 'Auto', 'Cashing a check without an account', 'Check cashing', 'Checking account', 'Conventional adjustable mortgage (ARM)', 'Conventional fixed mortgage', 'Credit card', 'Credit repair', 'Debt settlement', 'Domestic (US) money transfer', 'Electronic Benefit Transfer / EBT card', 'FHA mortgage', 'Federal student loan', 'Foreign currency exchange', 'General purpose card', 'Gift or merchant card', 'Government benefit payment card', 'Home equity loan or line of credit', 'I do not know', 'ID prepaid card', 'Installment loan', 'International money transfer', 'Medical', 'Mobile wallet', 'Money order', 'Mortgage', 'Non-federal student loan', 'Other (i.e. phone, health club, etc.)', 'Other bank product/service', 'Other mortgage', 'Other special purpose card', 'Pawn loan', 'Payday loan', 'Payroll card', 'Personal line of credit', 'Refund anticipation check', 'Reverse mortgage', 'Savings account', 'Title loan', 'Transit card', 'Travelerâs/Cashierâs checks', 'VA mortgage', 'Vehicle lease', 'Vehicle loan'"
'issue',"'APR or interest rate', 'Account opening, closing, or management', 'Account terms and changes', 'Adding money', 'Advertising and marketing', 'Advertising, marketing or disclosures', 'Application processing delay', 'Application, originator, mortgage broker', 'Applied for loan/did not receive money', 'Arbitration', 'Balance transfer', 'Balance transfer fee', 'Bankruptcy', 'Billing disputes', 'Billing statement', 'Can\'t contact lender', 'Can\'t repay my loan', 'Can\'t stop charges to bank account', 'Cash advance', 'Cash advance fee', 'Charged bank acct wrong day or amt', 'Charged fees or interest I didn\'t expect', 'Closing/Cancelling account', 'Communication tactics', 'Cont\'d attempts collect debt not owed', 'Convenience checks', 'Credit card protection / Debt protection', 'Credit decision / Underwriting', 'Credit determination', 'Credit line increase/decrease', 'Credit monitoring or identity protection', 'Credit reporting company\'s investigation', 'Customer service / Customer relations', 'Customer service/Customer relations', 'Dealing with my lender or servicer', 'Delinquent account', 'Deposits and withdrawals', 'Disclosure verification of debt', 'Disclosures', 'Excessive fees', 'False statements or representation', 'Fees', 'Forbearance / Workout plans', 'Fraud or scam', 'Getting a loan', 'Identity theft / Fraud / Embezzlement', 'Improper contact or sharing of info', 'Improper use of my credit report', 'Incorrect exchange rate', 'Incorrect information on credit report', 'Incorrect/missing disclosures or info', 'Late fee', 'Lender damaged or destroyed vehicle', 'Lender repossessed or sold the vehicle', 'Lender sold the property', 'Loan modification,collection,foreclosure', 'Loan servicing, payments, escrow account', 'Lost or stolen check', 'Lost or stolen money order', 'Making/receiving payments, sending money', 'Managing the line of credit', 'Managing the loan or lease', 'Managing, opening, or closing account', 'Money was not available when promised', 'Other', 'Other fee', 'Other service issues', 'Other transaction issues', 'Overdraft, savings or rewards features', 'Overlimit fee', 'Payment to acct not credited', 'Payoff process', 'Privacy', 'Problems caused by my funds being low', 'Problems when you are unable to pay', 'Received a loan I didn\'t apply for', 'Rewards', 'Sale of account', 'Settlement process and costs', 'Shopping for a line of credit', 'Shopping for a loan or lease', 'Taking out the loan or lease', 'Taking/threatening an illegal action', 'Transaction issue', 'Unable to get credit report/credit score', 'Unauthorized transactions/trans. issues', 'Unexpected/Other fees', 'Unsolicited issuance of credit card', 'Using a debit or ATM card', 

In [19]:
print(schema)

feature {
  name: "product"
  type: BYTES
  domain: "product"
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "sub_product"
  value_count {
    min: 1
    max: 1
  }
  type: BYTES
  domain: "sub_product"
  presence {
    min_count: 1
  }
}
feature {
  name: "issue"
  type: BYTES
  domain: "issue"
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "sub_issue"
  value_count {
    min: 1
    max: 1
  }
  type: BYTES
  domain: "sub_issue"
  presence {
    min_count: 1
  }
}
feature {
  name: "consumer_complaint_narrative"
  type: BYTES
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "company"
  type: BYTES
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "state"
  value_count {
    min: 1
    max: 1
  }
  type:

## Save the TFDV schema


In [20]:

base_dir = pwd.getpwuid(os.getuid()).pw_dir
#base_dir = os.getcwd()
data_dir_str = 'Github/building-machine-learning-pipelines/data-output/chap4/schema'
try:
    os.mkdir(data_dir)
except:
    print('dir already exists')

data_dir = os.path.join(base_dir, data_dir_str)
tfdv.write_schema_text(schema, output_path)

## Comparing Datasets
The following code section shows how to compare two statistics for tfrecords.  It will fail unless you have tfrecord files. 



In [ ]:
lhstats = tfdv.generate_statistics_from_tfrecord(data_location=os.path.join(data_dir, 'consumer_complaints_with_narrative.tfrecord'))
rhstats = tfdv.generate_statistics_from_tfrecord(data_location=os.path.join(data_dir, 'consumer_complaints_with_narrative2.tfrecord'))

tfdv.vistualie_statistics(lhs_statistics=val_stats,rhs_statistics=train_stats, lhs_name='VAL_DATASET', rhs_name='TRAIN_DATASET')

## Detecting anomalies

In [18]:
anomalies = tfdv.validate_statistics(statistics=stats, schema=schema)

tfdv.display_anomalies(anomalies)

## Updating the schema
If you need to modify the schema that was autogenerate above, then you can modify it with the code below.

In [ ]:
base_dir = pwd.getpwuid(os.getuid()).pw_dir
data_dir_str = 'Github/building-machine-learning-pipelines/data-output/chap4/schema'
schema_dir = os.path.join(base_dir, data_dir_str)
schema = tfdv.load_schema_text(schema_dir)

In [ ]:
sub_issue_feature = tfdv.get_feature(schema, 'sub_issue')
sub_issue_feature.presence.min_fraction = 0.9
state_domain = tfdv.get_domain(schema, 'state')
state_domain.value.remove('AK')

tfdv.write_schema_text(schema, schema_dir)